In [57]:
from geopandas import GeoDataFrame
from shapely.geometry import Point
import geemap
import ee
import os
import geopandas as gpd

from shapely import wkt
import pandas as pd
import matplotlib.pyplot as plt

### Funciones

In [58]:
def calcular_distancia(imagen_pixel, nom_vars_list, df_sh_list):
    
    """Funcion para calcular la distancia de la imagen con diferentes sectores (medido en metros)"""
    
    for i in range(len(df_sh_list)):
        print('revisando ' + str(i) + '----')
        df_sh_list[i].to_crs(epsg=32719, inplace=True)                # to meters
        imagen_pixel[nom_vars_list[i]] = imagen_pixel.geometry.apply(lambda x: df_sh_list[i].distance(x).min())  # add distance colums
    
    return imagen_pixel

def add_time(image):
    """Funcion para realizar un filtro de fechas con map"""
    return image.set('Date', image.date().format("YYYY-MM-dd"))



def calculate_weather(image, fecha, df_4326):
    
    # filtro df
    df_aux = df_4326[df_4326['time2'] == fecha] # global env
    
    # recolectamos los puntos
    points = df_aux.geometry

    # formato apto para buscar los puntos en gee
    jsonDict = eval(points.to_json()) 
    fc = ee.FeatureCollection(jsonDict)
    imagen_per_point = image.sampleRegions(collection = fc)
    
    fc_point_samp = geemap.ee_to_df(imagen_per_point)
    df_aux = df_aux.reset_index(drop= True)
    fc_point_samp = fc_point_samp.reset_index(drop= True)
    
    return pd.concat([df_aux, fc_point_samp], axis=1)


### Estimando distancias

In [67]:
# muestra_datos_consolidados_v2 = pd.read_csv('muestra_datos_consolidados_v2.csv')
muestra_datos_consolidados_v2 = pd.read_csv('../model/data/valparaiso/datos_consolidados_sentinel2.csv')
muestra_datos_consolidados_v2

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,comuna,codreg,codprov,codcom,nom_incen,causa,superficie,fecha_ini,fecha_ter,tipo_dato
0,0,2022-02-02,0,0.1477,0.1538,0.1772,0.2068,0.3074,0.4081,0.3386,...,San Antonio,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio
1,1,2022-02-02,1,0.1337,0.1473,0.1659,0.1939,0.2746,0.3788,0.2957,...,San Antonio,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio
2,2,2022-02-02,2,0.1343,0.1363,0.1546,0.1620,0.2888,0.2992,0.2304,...,San Antonio,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio
3,3,2022-02-02,3,0.1418,0.1545,0.1804,0.2299,0.3376,0.4248,0.3199,...,San Antonio,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio
4,4,2022-02-02,4,0.1412,0.1424,0.1610,0.1752,0.2749,0.3395,0.2616,...,San Antonio,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20284,11284,2020-12-24,285,0.0809,0.0949,0.1236,0.1616,0.2426,0.2465,0.2078,...,Quilpu.,5,58,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random
20285,11285,2020-12-24,286,0.0580,0.0758,0.0941,0.1166,0.1865,0.1929,0.1561,...,Quilpu.,5,58,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random
20286,11286,2020-12-24,292,0.0580,0.0621,0.0887,0.1009,0.2713,0.2881,0.2071,...,Quilpu.,5,58,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random
20287,11287,2020-12-24,296,0.0680,0.0889,0.1179,0.1594,0.2134,0.2131,0.1751,...,Quilpu.,5,58,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random


In [68]:
geometry = [Point(xy) for xy in zip(muestra_datos_consolidados_v2.x, muestra_datos_consolidados_v2.y)]
# df = muestra_datos_consolidados_v2.drop(['x', 'y'], axis=1)
df = muestra_datos_consolidados_v2
df = GeoDataFrame(df, crs="EPSG:32719", geometry=geometry)
df

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,codreg,codprov,codcom,nom_incen,causa,superficie,fecha_ini,fecha_ter,tipo_dato,geometry
0,0,2022-02-02,0,0.1477,0.1538,0.1772,0.2068,0.3074,0.4081,0.3386,...,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (269510.000 6265950.000)
1,1,2022-02-02,1,0.1337,0.1473,0.1659,0.1939,0.2746,0.3788,0.2957,...,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (270490.000 6263510.000)
2,2,2022-02-02,2,0.1343,0.1363,0.1546,0.1620,0.2888,0.2992,0.2304,...,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (268110.000 6263470.000)
3,3,2022-02-02,3,0.1418,0.1545,0.1804,0.2299,0.3376,0.4248,0.3199,...,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (268050.000 6266010.000)
4,4,2022-02-02,4,0.1412,0.1424,0.1610,0.1752,0.2749,0.3395,0.2616,...,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (268590.000 6265750.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20284,11284,2020-12-24,285,0.0809,0.0949,0.1236,0.1616,0.2426,0.2465,0.2078,...,5,58,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random,POINT (364170.000 6358850.000)
20285,11285,2020-12-24,286,0.0580,0.0758,0.0941,0.1166,0.1865,0.1929,0.1561,...,5,58,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random,POINT (335850.000 6395730.000)
20286,11286,2020-12-24,292,0.0580,0.0621,0.0887,0.1009,0.2713,0.2881,0.2071,...,5,58,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random,POINT (285190.000 6328590.000)
20287,11287,2020-12-24,296,0.0680,0.0889,0.1179,0.1594,0.2134,0.2131,0.1751,...,5,58,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random,POINT (339410.000 6401790.000)


In [69]:
df.crs

<Projected CRS: EPSG:32719>
Name: WGS 84 / UTM zone 19S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 72°W and 66°W, southern hemisphere between 80°S and equator, onshore and offshore. Argentina. Bolivia. Brazil. Chile. Colombia. Peru.
- bounds: (-72.0, -80.0, -66.0, 0.0)
Coordinate Operation:
- name: UTM zone 19S
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
# load data pixeles
# df = pd.read_csv('muestra_datos_consolidados_v2.csv')

# # editando geometria
# df['geometry'] = df['geometry'].apply(wkt.loads)
# df = gpd.GeoDataFrame(df, geometry='geometry')
# df.crs = "EPSG:32719"

In [70]:
# load .shp
# lista_gdfs = []
dct_gdfs = {
    "gdf": [],
    "ruta": []
}

for directorio_raiz, directorios, archivos in os.walk('../shp'):
    for i, archivo in enumerate(archivos):
        if archivo.endswith('.shp'):            # Solo considerar archivos con extensión .shp
            ruta_archivo = os.path.join(directorio_raiz, archivo)
            print(i, ruta_archivo)
            gdf = gpd.read_file(ruta_archivo)   # Cargar el Shapefile
            # lista_gdfs.append(gdf)
            dct_gdfs["gdf"].append(gdf)
            dct_gdfs["ruta"].append(ruta_archivo)

# print(lista_gdfs)

1 ../shp/red_hidrografica/Red_Hidrografica.shp
5 ../shp/red_vial/redvial2019.shp
6 ../shp/areas_silverstres_protegidas/snaspe.shp
5 ../shp/areas_urbanas/Areas_Pobladas.shp
1 ../shp/red_ferroviaria/Red_ferroviaria.shp
0 ../shp/Masas_lacustres_sector_sur/Masas_lacustres_sector_sur.shp
7 ../shp/masas_lacustres/masas_lacustres.shp


In [72]:
for k in dct_gdfs:
    print(k, len(dct_gdfs[k]))

for i, r in enumerate(dct_gdfs["ruta"]):
    print(i, r)

gdf 7
ruta 7
0 ../shp/red_hidrografica/Red_Hidrografica.shp
1 ../shp/red_vial/redvial2019.shp
2 ../shp/areas_silverstres_protegidas/snaspe.shp
3 ../shp/areas_urbanas/Areas_Pobladas.shp
4 ../shp/red_ferroviaria/Red_ferroviaria.shp
5 ../shp/Masas_lacustres_sector_sur/Masas_lacustres_sector_sur.shp
6 ../shp/masas_lacustres/masas_lacustres.shp


In [73]:
# lista_gdfs[0]
dct_gdfs["gdf"][5]

,OBJECTID,Nombre,Tipo,Observacio,REGION,PROVINCIA,COMUNA,CATEGORIA,SUBCATEGOR,AREA_MT2,ANEXO,Shape__Are,Shape__Len,geometry
0,1,Tranque Jardín Botánico,Tranque,NaN,V region,Valparaíso,Viña del Mar,Medioambiente,Hidrografía,6.116670e+03,NaN,6.116670e+03,479.988067,"POLYGON ((-71.49660 -33.03909, -71.49663 -33.0..."
1,2,SN,Tranque,Area reducida: mediante fotointerpretación,V region,San Antonio,Algarrobo,Medioambiente,Hidrografía,6.406697e+03,NaN,6.406698e+03,359.366997,"POLYGON ((-71.61045 -33.31400, -71.61049 -33.3..."
2,3,SN,Tranque,NaN,V region,Valparaíso,Casablanca,Medioambiente,Hidrografía,7.212362e+03,NaN,7.212363e+03,519.104631,"POLYGON ((-71.67055 -33.18685, -71.67050 -33.1..."
3,4,SN,Tranque,NaN,V region,Valparaíso,Valparaíso,Medioambiente,Hidrografía,1.425137e+04,NaN,1.425137e+04,529.395140,"POLYGON ((-71.64097 -33.06775, -71.64096 -33.0..."
4,5,SN,Tranque,NaN,V region,San Antonio,Santo Domingo,Medioambiente,Hidrografía,3.690565e+04,NaN,3.690565e+04,1118.674237,"POLYGON ((-71.75298 -33.88001, -71.75284 -33.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,85,Embalse San Juan,Embalse,Area reducida: mediante fotointerpretación,V region,San Antonio,San Antonio,Medioambiente,Hidrografía,1.792888e+05,NaN,1.792888e+05,2097.659681,"POLYGON ((-71.53861 -33.63210, -71.53879 -33.6..."
85,86,Embalse Los Molles,Embalse,Seco: mediante fotointerpretación,V region,San Antonio,Santo Domingo,Medioambiente,Hidrografía,1.016837e+06,NaN,1.016837e+06,8277.432085,"POLYGON ((-71.68482 -33.79259, -71.68558 -33.7..."
86,87,Embalse Parque,Embalse,NaN,V region,San Antonio,Santo Domingo,Medioambiente,Hidrografía,1.070249e+05,NaN,1.070248e+05,1311.170909,"POLYGON ((-71.66753 -33.85607, -71.66716 -33.8..."
87,88,Embalse Los Pozos,Embalse,Area reducida: mediante fotointerpretación,V region,San Antonio,Santo Domingo,Medioambiente,Hidrografía,7.399300e+04,NaN,7.399300e+04,1379.854604,"POLYGON ((-71.70404 -33.89033, -71.70424 -33.8..."


NO SIRVEN:

- shp\masas_lacustres\masas_lacustres.shp
- shp\red_ferroviaria\Red_ferroviaria.shp 


In [74]:
# areas_silvestres_protegidas = lista_gdfs[0][lista_gdfs[0]['Region'] == 'Región de Valparaíso']
# red_hidrografica = lista_gdfs[5][lista_gdfs[5]['Region'] == 'Región de Valparaíso']

areas_silvestres_protegidas = dct_gdfs["gdf"][2][dct_gdfs["gdf"][2]['Region'] == 'Región de Valparaíso']
red_hidrografica = dct_gdfs["gdf"][0][dct_gdfs["gdf"][0]['Region'] == 'Región de Valparaíso']

In [75]:
areas_silvestres_protegidas
# red_hidrografica

,objectid,st_area_sh,st_length_,Nombre,Tipo_Snasp,Region,Cod_Region,geometry
28,2857,9.381431e+04,1964.761566,M.N.Isla Cachagua,Monumento,Región de Valparaíso,5,"POLYGON ((-7954624.290 -3840307.954, -7954610...."
84,2778,7.683010e+06,33304.765385,R.N.El Yali,Reserva,Región de Valparaíso,5,"POLYGON ((-7981984.242 -3994620.416, -7981651...."
100,390,1.085037e+08,48751.751393,P.N.La Campana,Parque,Región de Valparaíso,5,"POLYGON ((-7916836.722 -3888867.872, -7916556...."
102,371,1.386633e+08,60633.298373,R.N.Lago Peñuelas,Reserva,Región de Valparaíso,5,"POLYGON ((-7948449.211 -3922177.291, -7948449...."
133,12471,7.210095e+07,123858.614640,Parque Nacional Rapa Nui,Parque,Región de Valparaíso,5,"POLYGON ((-12177090.007 -3130867.041, -1217708..."
134,12473,1.455558e+07,15380.170438,Parque Nacional Rapa Nui,Parque,Región de Valparaíso,5,"POLYGON ((-12182937.223 -3143895.870, -1218270..."
135,12472,1.234812e+06,4720.500853,Parque Nacional Rapa Nui,Parque,Región de Valparaíso,5,"POLYGON ((-12178144.007 -3141576.637, -1217817..."
136,12871,1.518290e+08,169912.258167,P.N. Juan Fernández,Parque,Región de Valparaíso,5,"MULTIPOLYGON (((-8993581.580 -4003769.913, -89..."


In [76]:
# AREAS URBANAS VALPO

areas_urbanas = dct_gdfs["gdf"][3]

aux = red_hidrografica['Comuna'].unique()  # no esta rapa nui ni juan fernandez!
print(aux)
aux[1] = 'La Calera'

# areas_urbanas = lista_gdfs[1][lista_gdfs[1]['comuna'].isin(aux)]
areas_urbanas = areas_urbanas[areas_urbanas['comuna'].isin(aux)]
areas_urbanas

['Panquehue' 'Calera' 'La Cruz' 'Rinconada' 'San Felipe' 'Llaillay'
 'Petorca' 'La Ligua' 'Cabildo' 'Olmué' 'Viña del Mar' 'Hijuelas'
 'Limache' 'Zapallar' 'Nogales' 'Catemu' 'Puchuncaví' 'Putaendo'
 'Quillota' 'Concón' 'Papudo' 'Quintero' 'Algarrobo' 'Santa María'
 'Calle Larga' 'Villa Alemana' 'Quilpué' 'Valparaíso' 'Casablanca'
 'El Quisco' 'El Tabo' 'Cartagena' 'Santo Domingo' 'San Antonio'
 'San Esteban' 'Los Andes']


,objectid,comuna,st_area_sh,st_length_,Localidad,Entidad,geometry
158,194,Casablanca,4.021006e+05,4018.578534,Quintay,Pueblo,"POLYGON ((-7981495.070 -3920944.993, -7981491...."
195,193,Algarrobo,2.950864e+05,3218.526174,El Yeco,Pueblo,"POLYGON ((-7976028.683 -3936619.030, -7975974...."
198,195,Quillota,8.630518e+05,7120.219432,San Pedro,Pueblo,"POLYGON ((-7934669.810 -3887276.826, -7934659...."
199,196,Zapallar,7.070413e+05,3922.796333,Catapilco,Pueblo,"POLYGON ((-7934702.603 -3837693.576, -7934687...."
202,198,Cabildo,2.800146e+06,11954.646432,Cabildo,Ciudad,"POLYGON ((-7911753.269 -3819690.485, -7911593...."
203,202,Putaendo,1.594487e+06,6567.120103,Putaendo,Ciudad,"POLYGON ((-7871991.423 -3846942.932, -7872231...."
204,200,Petorca,1.562601e+06,9892.170654,Petorca,Pueblo,"POLYGON ((-7897938.923 -3796707.681, -7897927...."
205,201,La Ligua,5.086825e+06,17968.319378,La Ligua,Ciudad,"POLYGON ((-7929150.053 -3823283.859, -7929152...."
207,203,San Felipe,1.250840e+07,27539.938175,San Felipe,Ciudad,"POLYGON ((-7875156.498 -3860508.871, -7874746...."
208,207,Llaillay,3.808810e+06,16938.978687,Llaillay,Ciudad,"POLYGON ((-7898745.990 -3874524.009, -7899657...."


In [77]:
# red_vial_2019 = lista_gdfs[6][lista_gdfs[6]['Cod_Region']==5]
red_vial_2019 = dct_gdfs["gdf"][1][dct_gdfs["gdf"][1]['Cod_Region']==5]
red_vial_2019

,shape_leng,st_length_,Nom_Ruta,Rol_Mop,Clase_Ruta,Tipo_Carpe,Catego,Cod_Region,geometry
13,9603.351160,937.122567,NaN,E-105-G,3,Tierra,Rural,5,"LINESTRING (339118.491 6353577.074, 339155.396..."
348,33.531712,235.546826,NaN,NaN,9,NaN,Urbano,5,"LINESTRING (317043.464 6371100.066, 317109.934..."
349,33.531712,48.546574,NaN,NaN,9,NaN,Urbano,5,"LINESTRING (316634.322 6371177.909, 316637.778..."
350,33.531712,52.984459,NaN,NaN,9,NaN,Urbano,5,"LINESTRING (316589.890 6371172.969, 316594.894..."
351,33.531712,63.491028,NaN,NaN,9,NaN,Urbano,5,"LINESTRING (316544.705 6371167.946, 316551.152..."
...,...,...,...,...,...,...,...,...,...
434783,1885.731382,2026.357795,NaN,E-895,3,Pavimento,Rural,5,"LINESTRING (351460.685 6362005.815, 351452.691..."
434784,755.955997,755.955997,Pascual Baburizza,NaN,3,NaN,Rural,5,"LINESTRING (351291.864 6363100.215, 351293.748..."
434785,0.000000,650.347086,Cruce Ruta 57 - Badén La Caldera - Cruce E-909,S/R-E-917,3,NaN,Urbano,5,"LINESTRING (349063.448 6360443.645, 349078.885..."
434786,0.000000,571.464714,Cruce Ruta 57 - Badén La Caldera - Cruce E-909,S/R-E-917,3,NaN,Rural,5,"LINESTRING (348982.469 6359902.944, 348982.789..."


#### Calculando distancias:

In [78]:
masas_lacustres_sur = dct_gdfs["gdf"][5]

nom_vars_list = ['d_areas_silvestres_protegidas', 'd_red_hidrografica', 'd_areas_urbanas', 'd_red_vial', 'd_masas_lacustres_sur']
df_sh_list = [areas_silvestres_protegidas, red_hidrografica, areas_urbanas, red_vial_2019, masas_lacustres_sur]

df_dist = calcular_distancia(df, nom_vars_list, df_sh_list)
df_dist

revisando 0----


/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


revisando 1----


/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


revisando 2----


...

revisando 3----


/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


revisando 4----


/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,superficie,fecha_ini,fecha_ter,tipo_dato,geometry,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial,d_masas_lacustres_sur
0,0,2022-02-02,0,0.1477,0.1538,0.1772,0.2068,0.3074,0.4081,0.3386,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (269510.000 6265950.000),16902.677963,400.154753,12126.140357,2294.313820,2434.561273
1,1,2022-02-02,1,0.1337,0.1473,0.1659,0.1939,0.2746,0.3788,0.2957,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (270490.000 6263510.000),17237.394379,453.269986,14583.117701,1772.619230,1862.415393
2,2,2022-02-02,2,0.1343,0.1363,0.1546,0.1620,0.2888,0.2992,0.2304,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (268110.000 6263470.000),14893.261402,72.702585,13066.062044,4135.188890,575.876133
3,3,2022-02-02,3,0.1418,0.1545,0.1804,0.2299,0.3376,0.4248,0.3199,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (268050.000 6266010.000),15549.403654,726.001225,11098.218300,3294.183857,1733.838615
4,4,2022-02-02,4,0.1412,0.1424,0.1610,0.1752,0.2749,0.3395,0.2616,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (268590.000 6265750.000),15967.776971,798.530491,11645.991187,3041.287838,1692.596209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20284,11284,2020-12-24,285,0.0809,0.0949,0.1236,0.1616,0.2426,0.2465,0.2078,...,348.660,2020-12-15,2020-12-23,zona_random,POINT (364170.000 6358850.000),52212.417080,377.329445,12995.937345,4461.495540,86530.974809
20285,11285,2020-12-24,286,0.0580,0.0758,0.0941,0.1166,0.1865,0.1929,0.1561,...,348.660,2020-12-15,2020-12-23,zona_random,POINT (335850.000 6395730.000),49242.351765,690.433235,6797.837863,1341.975167,79805.605778
20286,11286,2020-12-24,292,0.0580,0.0621,0.0887,0.1009,0.2713,0.2881,0.2071,...,348.660,2020-12-15,2020-12-23,zona_random,POINT (285190.000 6328590.000),7939.849323,111.976747,11425.458840,243.862443,7200.566948
20287,11287,2020-12-24,296,0.0680,0.0889,0.1179,0.1594,0.2134,0.2131,0.1751,...,348.660,2020-12-15,2020-12-23,zona_random,POINT (339410.000 6401790.000),56264.995966,1289.604321,12153.876026,9.022501,86041.906391


In [79]:
# del lista_gdfs
df_dist.describe()

,Unnamed: 0,dim_0,coastal,blue,green,red,nir08,swir16,swir22,y,...,id_pixel_incendio,codreg,codprov,codcom,superficie,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial,d_masas_lacustres_sur
count,20289.000000,20289.000000,19377.000000,19374.000000,19377.000000,19384.000000,19389.000000,19396.000000,19396.000000,2.028900e+04,...,20289.000000,20289.0,20289.000000,20289.000000,20289.000000,20289.000000,20289.000000,20289.000000,20289.000000,20289.000000
mean,5136.311105,103.713391,0.072547,0.081475,0.102068,0.121793,0.221108,0.238231,0.182321,6.349209e+06,...,278.187934,5.0,54.967076,5499.047760,676.968192,28212.407268,602.428159,11899.841053,3550.065777,39329.569938
std,3037.619689,82.821514,0.089048,0.085610,0.082812,0.086879,0.084520,0.082264,0.070688,5.314956e+04,...,294.121008,0.0,2.704291,270.332069,788.619348,25084.032171,518.325203,9584.902052,5466.813820,39605.566253
min,0.000000,0.000000,0.000000,0.000100,0.000000,0.000100,0.000000,0.000100,0.000100,6.241570e+06,...,1.000000,5.0,51.000000,5101.000000,210.938000,0.000000,0.118358,0.000000,0.024954,0.000000
25%,2536.000000,39.000000,0.035300,0.042300,0.060400,0.070100,0.173300,0.182000,0.132800,6.323030e+06,...,96.000000,5.0,51.000000,5105.000000,252.660000,8865.777847,215.057025,5056.501971,615.529038,3950.154452
50%,5072.000000,78.000000,0.050700,0.062000,0.084100,0.109100,0.214400,0.230600,0.178400,6.347030e+06,...,192.000000,5.0,56.000000,5601.000000,336.354000,20079.905199,471.146084,9276.949148,1666.874133,21789.364157
75%,7608.000000,158.000000,0.075300,0.088900,0.115900,0.149225,0.256600,0.290400,0.223225,6.392730e+06,...,367.000000,5.0,58.000000,5801.000000,563.156816,39867.927275,853.893249,17600.719411,4175.091601,73338.112405
max,11288.000000,299.000000,0.999600,0.998600,0.983900,0.998800,0.996700,0.948900,0.815200,6.454650e+06,...,1808.000000,5.0,58.000000,5804.000000,3403.060000,99474.046340,4652.871557,54056.892229,38161.298928,134097.330190


In [80]:
df_dist.shape, df.shape

((20289, 36), (20289, 36))

### Datos climáticos con Google Earth Engine

In [49]:
import ee
ee.Authenticate()


Successfully saved authorization token.


In [50]:
# Inicializar Earth Engine
ee.Initialize()

In [81]:
df

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,superficie,fecha_ini,fecha_ter,tipo_dato,geometry,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial,d_masas_lacustres_sur
0,0,2022-02-02,0,0.1477,0.1538,0.1772,0.2068,0.3074,0.4081,0.3386,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (269510.000 6265950.000),16902.677963,400.154753,12126.140357,2294.313820,2434.561273
1,1,2022-02-02,1,0.1337,0.1473,0.1659,0.1939,0.2746,0.3788,0.2957,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (270490.000 6263510.000),17237.394379,453.269986,14583.117701,1772.619230,1862.415393
2,2,2022-02-02,2,0.1343,0.1363,0.1546,0.1620,0.2888,0.2992,0.2304,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (268110.000 6263470.000),14893.261402,72.702585,13066.062044,4135.188890,575.876133
3,3,2022-02-02,3,0.1418,0.1545,0.1804,0.2299,0.3376,0.4248,0.3199,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (268050.000 6266010.000),15549.403654,726.001225,11098.218300,3294.183857,1733.838615
4,4,2022-02-02,4,0.1412,0.1424,0.1610,0.1752,0.2749,0.3395,0.2616,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (268590.000 6265750.000),15967.776971,798.530491,11645.991187,3041.287838,1692.596209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20284,11284,2020-12-24,285,0.0809,0.0949,0.1236,0.1616,0.2426,0.2465,0.2078,...,348.660,2020-12-15,2020-12-23,zona_random,POINT (364170.000 6358850.000),52212.417080,377.329445,12995.937345,4461.495540,86530.974809
20285,11285,2020-12-24,286,0.0580,0.0758,0.0941,0.1166,0.1865,0.1929,0.1561,...,348.660,2020-12-15,2020-12-23,zona_random,POINT (335850.000 6395730.000),49242.351765,690.433235,6797.837863,1341.975167,79805.605778
20286,11286,2020-12-24,292,0.0580,0.0621,0.0887,0.1009,0.2713,0.2881,0.2071,...,348.660,2020-12-15,2020-12-23,zona_random,POINT (285190.000 6328590.000),7939.849323,111.976747,11425.458840,243.862443,7200.566948
20287,11287,2020-12-24,296,0.0680,0.0889,0.1179,0.1594,0.2134,0.2131,0.1751,...,348.660,2020-12-15,2020-12-23,zona_random,POINT (339410.000 6401790.000),56264.995966,1289.604321,12153.876026,9.022501,86041.906391


In [82]:
df_4326 = df.to_crs("EPSG:4326")
df_4326['time2'] = [item.split(" ", 1)[0] for item in df_4326['time']]

# Obtenemos las fechas de captura de cada uno, para realizar un posterior filtro en las imagenes recolectadas
DATES = df_4326.time.unique()
DATES = [item.split(" ", 1)[0] for item in DATES]
print(len(DATES))
print(DATES)

inicio_datetime = min(DATES)
fin_datetime = max(DATES)

53
['2022-02-02', '2022-02-07', '2022-02-12', '2022-02-17', '2022-02-22', '2022-02-27', '2022-03-04', '2021-07-02', '2021-07-07', '2021-07-17', '2021-07-22', '2021-07-27', '2021-08-01', '2022-03-09', '2017-01-19', '2017-01-29', '2017-02-18', '2017-02-28', '2017-03-10', '2017-03-20', '2018-12-10', '2018-12-15', '2018-12-20', '2018-12-30', '2019-01-04', '2019-01-14', '2019-01-19', '2019-01-24', '2019-01-29', '2019-02-03', '2019-01-09', '2022-12-04', '2022-12-09', '2022-12-14', '2022-12-19', '2022-12-24', '2023-05-03', '2023-05-08', '2023-05-18', '2022-12-29', '2022-10-30', '2022-11-04', '2022-11-19', '2020-12-24', '2020-12-29', '2021-01-03', '2021-01-13', '2021-01-18', '2020-11-24', '2020-11-29', '2020-12-04', '2020-12-09', '2020-12-19']


In [83]:
sorted(DATES)

['2017-01-19',
 '2017-01-29',
 '2017-02-18',
 '2017-02-28',
 '2017-03-10',
 '2017-03-20',
 '2018-12-10',
 '2018-12-15',
 '2018-12-20',
 '2018-12-30',
 '2019-01-04',
 '2019-01-09',
 '2019-01-14',
 '2019-01-19',
 '2019-01-24',
 '2019-01-29',
 '2019-02-03',
 '2020-11-24',
 '2020-11-29',
 '2020-12-04',
 '2020-12-09',
 '2020-12-19',
 '2020-12-24',
 '2020-12-29',
 '2021-01-03',
 '2021-01-13',
 '2021-01-18',
 '2021-07-02',
 '2021-07-07',
 '2021-07-17',
 '2021-07-22',
 '2021-07-27',
 '2021-08-01',
 '2022-02-02',
 '2022-02-07',
 '2022-02-12',
 '2022-02-17',
 '2022-02-22',
 '2022-02-27',
 '2022-03-04',
 '2022-03-09',
 '2022-10-30',
 '2022-11-04',
 '2022-11-19',
 '2022-12-04',
 '2022-12-09',
 '2022-12-14',
 '2022-12-19',
 '2022-12-24',
 '2022-12-29',
 '2023-05-03',
 '2023-05-08',
 '2023-05-18']

In [84]:
df_4326

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,fecha_ini,fecha_ter,tipo_dato,geometry,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial,d_masas_lacustres_sur,time2
0,0,2022-02-02,0,0.1477,0.1538,0.1772,0.2068,0.3074,0.4081,0.3386,...,2022-02-23,2022-03-02,zona_incendio,POINT (-71.48750 -33.72145),16902.677963,400.154753,12126.140357,2294.313820,2434.561273,2022-02-02
1,1,2022-02-02,1,0.1337,0.1473,0.1659,0.1939,0.2746,0.3788,0.2957,...,2022-02-23,2022-03-02,zona_incendio,POINT (-71.47756 -33.74365),17237.394379,453.269986,14583.117701,1772.619230,1862.415393,2022-02-02
2,2,2022-02-02,2,0.1343,0.1363,0.1546,0.1620,0.2888,0.2992,0.2304,...,2022-02-23,2022-03-02,zona_incendio,POINT (-71.50324 -33.74349),14893.261402,72.702585,13066.062044,4135.188890,575.876133,2022-02-02
3,3,2022-02-02,3,0.1418,0.1545,0.1804,0.2299,0.3376,0.4248,0.3199,...,2022-02-23,2022-03-02,zona_incendio,POINT (-71.50322 -33.72059),15549.403654,726.001225,11098.218300,3294.183857,1733.838615,2022-02-02
4,4,2022-02-02,4,0.1412,0.1424,0.1610,0.1752,0.2749,0.3395,0.2616,...,2022-02-23,2022-03-02,zona_incendio,POINT (-71.49747 -33.72305),15967.776971,798.530491,11645.991187,3041.287838,1692.596209,2022-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20284,11284,2020-12-24,285,0.0809,0.0949,0.1236,0.1616,0.2426,0.2465,0.2078,...,2020-12-15,2020-12-23,zona_random,POINT (-70.45234 -32.90012),52212.417080,377.329445,12995.937345,4461.495540,86530.974809,2020-12-24
20285,11285,2020-12-24,286,0.0580,0.0758,0.0941,0.1166,0.1865,0.1929,0.1561,...,2020-12-15,2020-12-23,zona_random,POINT (-70.74853 -32.56371),49242.351765,690.433235,6797.837863,1341.975167,79805.605778,2020-12-24
20286,11286,2020-12-24,292,0.0580,0.0621,0.0887,0.1009,0.2713,0.2881,0.2071,...,2020-12-15,2020-12-23,zona_random,POINT (-71.30342 -33.16020),7939.849323,111.976747,11425.458840,243.862443,7200.566948,2020-12-24
20287,11287,2020-12-24,296,0.0680,0.0889,0.1179,0.1594,0.2134,0.2131,0.1751,...,2020-12-15,2020-12-23,zona_random,POINT (-70.70958 -32.50959),56264.995966,1289.604321,12153.876026,9.022501,86041.906391,2020-12-24


In [86]:
df_4326[df_4326["time"] == "2017-01-19"]

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,fecha_ini,fecha_ter,tipo_dato,geometry,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial,d_masas_lacustres_sur,time2
1600,1600,2017-01-19,0,0.0631,0.0622,0.0873,0.1127,0.1951,0.3252,0.2476,...,2017-01-13,2017-01-17,zona_incendio,POINT (-71.32876 -32.99905),17511.649013,312.422800,4140.166815,52.434204,11413.570353,2017-01-19
1601,1601,2017-01-19,1,0.0609,0.0614,0.0827,0.0955,0.1866,0.2415,0.1716,...,2017-01-13,2017-01-17,zona_incendio,POINT (-71.35225 -33.01321),16469.175068,331.000077,2151.869842,56.939837,9807.945301,2017-01-19
1602,1602,2017-01-19,2,0.0499,0.0426,0.0516,0.0580,0.0794,0.1310,0.1406,...,2017-01-13,2017-01-17,zona_incendio,POINT (-71.34343 -33.00328),17826.073400,14.929975,3394.308556,972.366614,10201.239306,2017-01-19
1603,1603,2017-01-19,3,0.0622,0.0635,0.0839,0.1038,0.2125,0.2971,0.2056,...,2017-01-13,2017-01-17,zona_incendio,POINT (-71.32718 -33.00394),17433.827330,44.272616,3940.201090,97.235166,11684.146353,2017-01-19
1604,1604,2017-01-19,4,0.0503,0.0467,0.0545,0.0571,0.0715,0.1087,0.1119,...,2017-01-13,2017-01-17,zona_incendio,POINT (-71.33702 -33.01206),17599.574900,463.203601,2755.770981,1029.052068,11077.508002,2017-01-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12654,3654,2017-01-19,285,0.0392,0.0476,0.0661,0.0732,0.2291,0.2063,0.1223,...,2017-01-09,2017-01-14,zona_random,POINT (-71.54421 -33.16778),0.000000,196.285437,2663.933764,144.597573,1353.468789,2017-01-19
12655,3655,2017-01-19,287,0.0677,0.0914,0.1283,0.1524,0.2343,0.2360,0.1866,...,2017-01-09,2017-01-14,zona_random,POINT (-70.73398 -32.11474),85688.968256,63.422578,22700.847877,5647.125500,114085.031558,2017-01-19
12656,3656,2017-01-19,291,0.0582,0.0714,0.1004,0.1175,0.1657,0.2001,0.1682,...,2017-01-09,2017-01-14,zona_random,POINT (-70.58844 -32.26584),85188.363222,1006.659455,30954.635568,14095.084140,111390.512917,2017-01-19
12657,3657,2017-01-19,292,0.1009,0.1219,0.1677,0.1960,0.2794,0.2892,0.2456,...,2017-01-09,2017-01-14,zona_random,POINT (-70.72896 -32.14097),84325.056646,586.192597,21308.746971,5213.666995,112110.866913,2017-01-19


In [87]:
list_df = []

for i, fecha in enumerate(DATES):
    print(f"Extrayendo info para el día: {(i, fecha)}")
    
    # Definir la colección de imágenes
    image_collection = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
        .filterDate(fecha) \
        .select([
            'temperature_2m',
            'temperature_2m_min',
            'temperature_2m_max',
            'dewpoint_temperature_2m',
            'dewpoint_temperature_2m_min',
            'dewpoint_temperature_2m_max',
            'soil_temperature_level_1',
            'soil_temperature_level_1_min',
            'soil_temperature_level_1_max',
            'volumetric_soil_water_layer_1',
            'volumetric_soil_water_layer_1_min',
            'volumetric_soil_water_layer_1_max',
            'u_component_of_wind_10m',
            'u_component_of_wind_10m_min',
            'u_component_of_wind_10m_max',
            'v_component_of_wind_10m',
            'v_component_of_wind_10m_min',
            'v_component_of_wind_10m_max',
            'total_precipitation_sum',
            'total_precipitation_min',
            'total_precipitation_max'
        ])
        
    # Realizando estimaciones: 
    df_aux = calculate_weather(image_collection.first(), fecha, df_4326)
              
    list_df.append(df_aux)
    
data_con_gee = pd.concat(list_df, ignore_index = True)
        

Extrayendo info para el día: (0, '2022-02-02')
Extrayendo info para el día: (1, '2022-02-07')
Extrayendo info para el día: (2, '2022-02-12')
Extrayendo info para el día: (3, '2022-02-17')
Extrayendo info para el día: (4, '2022-02-22')
Extrayendo info para el día: (5, '2022-02-27')
Extrayendo info para el día: (6, '2022-03-04')
Extrayendo info para el día: (7, '2021-07-02')
Extrayendo info para el día: (8, '2021-07-07')
Extrayendo info para el día: (9, '2021-07-17')
Extrayendo info para el día: (10, '2021-07-22')
Extrayendo info para el día: (11, '2021-07-27')
Extrayendo info para el día: (12, '2021-08-01')
Extrayendo info para el día: (13, '2022-03-09')
Extrayendo info para el día: (14, '2017-01-19')
Extrayendo info para el día: (15, '2017-01-29')
Extrayendo info para el día: (16, '2017-02-18')
Extrayendo info para el día: (17, '2017-02-28')
Extrayendo info para el día: (18, '2017-03-10')
Extrayendo info para el día: (19, '2017-03-20')
Extrayendo info para el día: (20, '2018-12-10')
Ex

In [88]:
data_con_gee

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,total_precipitation_sum,u_component_of_wind_10m,u_component_of_wind_10m_max,u_component_of_wind_10m_min,v_component_of_wind_10m,v_component_of_wind_10m_max,v_component_of_wind_10m_min,volumetric_soil_water_layer_1,volumetric_soil_water_layer_1_max,volumetric_soil_water_layer_1_min
0,0,2022-02-02,0,0.1477,0.1538,0.1772,0.2068,0.3074,0.4081,0.3386,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
1,1,2022-02-02,1,0.1337,0.1473,0.1659,0.1939,0.2746,0.3788,0.2957,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
2,2,2022-02-02,2,0.1343,0.1363,0.1546,0.1620,0.2888,0.2992,0.2304,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
3,3,2022-02-02,3,0.1418,0.1545,0.1804,0.2299,0.3376,0.4248,0.3199,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
4,4,2022-02-02,4,0.1412,0.1424,0.1610,0.1752,0.2749,0.3395,0.2616,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20284,11159,2020-12-19,285,0.2138,0.2678,0.2331,0.1974,0.2564,0.2413,0.2716,...,8.532405e-05,0.892003,2.689728,-0.845230,0.142733,0.428673,-0.195715,0.152294,0.153549,0.149200
20285,11160,2020-12-19,286,0.0725,0.0769,0.0977,0.1191,0.1941,0.1895,0.1539,...,6.407499e-05,1.241247,3.500061,-0.789566,2.269788,3.112665,1.180629,0.250133,0.250488,0.250000
20286,11161,2020-12-19,292,0.0601,0.0578,0.0900,0.1103,0.2796,0.3053,0.2104,...,1.438856e-04,0.966100,2.930939,-0.879410,0.130912,0.608759,-0.520055,0.142519,0.143066,0.140961
20287,11162,2020-12-19,296,0.0786,0.0848,0.1085,0.1484,0.1816,0.1940,0.1692,...,3.752112e-05,1.135941,2.788147,-0.455582,0.356824,0.671259,-0.123822,0.149624,0.149796,0.149307


In [89]:
data_con_gee.dtypes

Unnamed: 0                              int64
time                                   object
dim_0                                   int64
coastal                               float64
blue                                  float64
green                                 float64
red                                   float64
nir08                                 float64
swir16                                float64
swir22                                float64
y                                     float64
x                                     float64
spatial_ref                             int64
es_incendio                             int64
id_pixel                                int64
id_incendio                            object
id_pixel_incendio                       int64
temporada                              object
region                                 object
provincia                              object
comuna                                 object
codreg                            

In [90]:
data_con_gee.to_csv('../model/data/valparaiso/data_consolidados_sentinel2_distancias_era5.csv')